In [1]:
import pandas as pd
from ntscraper import Nitter

In [2]:
Scraper = Nitter()

Testing instances: 100%|██████████| 77/77 [02:39<00:00,  2.08s/it]


In [83]:
#tweets =Scraper.get_tweets("realDonaldTrump",mode ='user',number = 100)
def scraping(input):
    tweets =Scraper.get_tweets(input,mode ='user',number = 500)
    return tweets

tweets = scraping("elonmusk")

24-Apr-24 03:22:03 - No instance specified, using random instance https://nitter.esmailelbob.xyz
24-Apr-24 03:22:10 - Current stats for elonmusk: 20 tweets, 0 threads...
24-Apr-24 03:22:15 - Current stats for elonmusk: 40 tweets, 0 threads...
24-Apr-24 03:22:21 - Current stats for elonmusk: 60 tweets, 0 threads...
24-Apr-24 03:22:26 - Current stats for elonmusk: 80 tweets, 0 threads...
24-Apr-24 03:22:31 - Current stats for elonmusk: 100 tweets, 0 threads...
24-Apr-24 03:22:35 - Current stats for elonmusk: 119 tweets, 0 threads...
24-Apr-24 03:22:40 - Current stats for elonmusk: 139 tweets, 0 threads...
24-Apr-24 03:22:44 - Current stats for elonmusk: 159 tweets, 0 threads...
24-Apr-24 03:22:49 - Current stats for elonmusk: 179 tweets, 0 threads...
24-Apr-24 03:22:54 - Current stats for elonmusk: 199 tweets, 0 threads...
24-Apr-24 03:23:00 - Current stats for elonmusk: 219 tweets, 0 threads...
24-Apr-24 03:23:05 - Current stats for elonmusk: 239 tweets, 0 threads...
24-Apr-24 03:23:10 

In [84]:
final_tweets = []
for tweet in tweets['tweets']:
    #data = [tweet['link'],tweet['text'],tweet['date'],tweet['stats']['likes'],tweet['stats']['comments']]
    data = [tweet['text'],tweet['date']]
    final_tweets.append(data)

In [106]:
data =pd.DataFrame(final_tweets, columns= ['text','date'])
#columns= ['link','text','date','no_likes','comment'])

In [107]:
data['date'] = pd.to_datetime(data['date'], format='%b %d, %Y · %I:%M %p %Z')
data['date'] = data['date'].dt.to_period('M')


/var/folders/wh/d15mczds4wxfm8qht9nw7p_w0000gn/T/ipykernel_43845/4118179080.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['date'] = data['date'].dt.to_period('M')


In [108]:
data.tail()

,text,date
495,,2024-03
496,Please send links from this platform to your f...,2024-03
497,Speech with full context,2024-03
498,Legacy media lies,2024-03
499,Easy to tell who is an NPC today,2024-03


In [109]:
import dill
def clean(text):
    import re
    import nltk
    from nltk.util import pr
    stemmer = nltk.SnowballStemmer("english")
    from nltk.corpus import stopwords
    import string
    stopword = set(stopwords.words("english"))
    text = str(text).lower()
    text = re.sub('[.?]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

with open('../backend/textback/models/clean_function.pkl', 'wb') as f:
     dill.dump(clean, f)

In [110]:
import pickle
model = pickle.load(open("../backend/textback/models/model.pkl","rb"))
cv = pickle.load(open("../backend/textback/models/vectorizer.pkl","rb"))

def predict(input):
    input = cv.transform([input]).toarray()
    predict = model.predict(input)[0]
    return predict

In [114]:
tweets_cleaned = []
Negative = 0
Nuetral = 0
for i in range(len(final_tweets)):
    clean_tweets = clean(final_tweets[i])
    if clean_tweets != "":
        tweets_cleaned.append(clean_tweets)


for i in range(len(tweets_cleaned)):
    prediction = predict(tweets_cleaned[i])
    if prediction == "No hate and offensive speech":
        Nuetral += 1
    elif prediction == "Offensive language detected":
        Negative += 1
    else :
       Negative += 1

#keep the score for each text in the dataframe file
data['score'] = data['text'].apply(predict)

#change score to numeric form 0,1,2
data['score'] = data['score'].replace({'No hate and offensive speech':0,'Offensive language detected':1,'Hate speech Detected':2})
print("Score about Negative and Nuetral in This user is\n Negative = " , Negative , "\n Nuetral = " , Nuetral)


Score about Negative and Nuetral in This user is
 Negative =  60 
 Nuetral =  440


In [115]:
#data to csv file
data['Month'] = data['date'].dt.month
data.to_csv('gay.csv',index=False)